# Data Description & Processing Methodology

This file further describes the nature of the data that I decided to analyze and the methods I used to efficiently 
manage and access the data in my analysis.  As discussed in README.md, I decided to analyze [the Medicare Part D prescriber data](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Part-D-Prescriber.html) for the years 2013, 2014, and 2015.

Now, let's take a look at the data to get an understanding of what information is provided and what information could be useful to examine.

In [1]:
import pandas as pd
import sqlalchemy as sqla
import sqlite3

years = [13, 14, 15]
MedData = pd.read_csv('MedicareData/PartD_Prescriber_PUF_NPI_Drug_15.txt', chunksize = 10000, 
                      low_memory= False, delimiter="\t")

MedChunk = next(MedData)
MedChunk.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_city', 'nppes_provider_state', 'specialty_description',
       'description_flag', 'drug_name', 'generic_name', 'bene_count',
       'total_claim_count', 'total_30_day_fill_count', 'total_day_supply',
       'total_drug_cost', 'bene_count_ge65', 'bene_count_ge65_suppress_flag',
       'total_claim_count_ge65', 'ge65_suppress_flag',
       'total_30_day_fill_count_ge65', 'total_day_supply_ge65',
       'total_drug_cost_ge65'],
      dtype='object')

In [2]:
MedChunk.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_city,nppes_provider_state,specialty_description,description_flag,drug_name,generic_name,bene_count,...,total_30_day_fill_count,total_day_supply,total_drug_cost,bene_count_ge65,bene_count_ge65_suppress_flag,total_claim_count_ge65,ge65_suppress_flag,total_30_day_fill_count_ge65,total_day_supply_ge65,total_drug_cost_ge65
0,1912175340,&H'S)U,&E'K:(A:I,SAN JUAN BAUTISTA,CA,General Surgery,S,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE/ACETAMINOPHEN,15.0,...,15.0,213,186.85,NaN,#,NaN,#,NaN,NaN,NaN
1,1912175340,&H'S)U,&E'K:(A:I,SAN JUAN BAUTISTA,CA,General Surgery,S,IBUPROFEN,IBUPROFEN,20.0,...,21.0,305,114.18,NaN,#,NaN,#,NaN,NaN,NaN
2,1912175340,&H'S)U,&E'K:(A:I,SAN JUAN BAUTISTA,CA,General Surgery,S,METOCLOPRAMIDE HCL,METOCLOPRAMIDE HCL,30.0,...,34.0,244,117.93,NaN,#,NaN,#,NaN,NaN,NaN
3,1417051921,A,N,CATOOSA,OK,Physician Assistant,S,ACETAMINOPHEN-CODEINE,ACETAMINOPHEN WITH CODEINE,17.0,...,17.0,44,57.64,NaN,*,NaN,*,NaN,NaN,NaN
4,1417051921,A,N,CATOOSA,OK,Physician Assistant,S,AZITHROMYCIN,AZITHROMYCIN,44.0,...,44.0,219,305.91,33.0,NaN,33.0,NaN,33.0,164.0,243.85


As we see above, each line in each of the data files corresponds to the information for a single drug provided by a single provider (i.e. nurse, doctor, specialist, etc.). Below are some important characteristics to note about the data in each file:
* Identifying information about each provider includes their National Provider Identifier (NPI number), their first and last name, their location (city and state), and their specialty and is repeated in every line regarding the provider.
* For any given drug and provider, the data set lists the official drug name, the generic name of the drug (if a generic drug, the drug name is the same as the generic name), the number of people for which the doctor prescribes the drug (`bene_count`), the number of claims made by patients of the provider for the drug including original scripts and refills (`total_claim_count`), and the total cost of each drug (includes the ingredient cost of the medication, dispensing fees, sales tax, and any applicable administration fees) paid by the Medicare Part D program (`total_drug_cost`).  
* The columns proceeded by `ge65_` correspond to the same information as described above, except only for the Medicare population that is greater than 65 years in age.
* To protect the personal health information (PHI) of the patients who use these doctors for treatment, the CMS omits the values for many of these variables if the number of patients that the provider is treating using the particular drug is below a certain threshold.  So, for example, if a provider only provided a single person `IBUPROFEN`, then most of the variables for that line item would be omitted.
* Additional information is available [through the Medicare Provider Utilization & Payment Data Methodology Overview](https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Prescriber_Methods.pdf).

Because I didn't want to introduce unnecessary bias into my analysis, I decided to only include variables that included information for every provider and drug combination. The most useful and intuitive information I found for every drug was provided by the `total_claim_count` and the `total_drug_cost` columns of the original dataset. Therefore, these are the only 2 numerical columns I included from the original data.

Also, I only wanted to include providers that had a reasonable looking name. Therefore, I excluded providers whose names did not meet the following criteria

1. Each provider name must be made of only alphabetic characters (no punctuation marks as we see in `MedChunk.head()` above.
2. The number of characters in a provider's full name (excluding spaces) must exceed 2. (For example, a provider whose first name is "N" and last name is "A" would not be included, but a provider whose first name is "AN" and whose last name is "B" would be).
3. The providers full name should not be None.

I wrote the function below, `wrangle_data`, to extract the necessary columns and rows from the original txt according to the above criteria.

In [3]:
### Read in data to an sql file (already done)
def wrangle_data(data_frame):
    """Keep Variables I want in the final sql file
    """
    vars_2_read = ['nppes_provider_first_name', 'nppes_provider_last_org_name', 'npi', 'nppes_provider_city',
               'nppes_provider_state', 'specialty_description', 'drug_name', 'generic_name', 
               'total_claim_count', 'total_drug_cost']
    new_cols = ['first_name', 'last_name', 'npi', 'city', 'state', 'specialty', 'drug_name', 'generic_name', 
                  'num_claims', 'cost']
    final_cols = ['name'] + new_cols[2:]
    
    data_frame = data_frame[vars_2_read].copy()
    data_frame.columns = new_cols
    data_frame['name'] = data_frame.first_name.str.cat(data_frame.last_name, sep = " ").str.title()
    
    # get rid of observations with invalid names
    invalid_names = ["None"]
    valid_names = ((data_frame.name.str.replace(" ", "").str.isalpha()) & # only alphabetic and spaces
                   (data_frame.name.str.replace(" ", "").str.len() > 2) & # names must be longer than 2
                   (~data_frame.name.isin(invalid_names)))                # no omitted names
    
    data_frame = data_frame[valid_names][final_cols]
    data_frame = data_frame.set_index('name')
    
    return data_frame

When applied to a pandas data frame of prescriber data, the function `wrangle_data` returns output in the form of below.

In [4]:
wrangle_data(MedChunk).head()

,npi,city,state,specialty,drug_name,generic_name,num_claims,cost
name,,,,,,,,
Barry Aab,1467482471,WAPAKONETA,OH,Dentist,AMOXICILLIN,AMOXICILLIN,19,54.04
Melanie Aaberg,1801093968,TRENTON,MI,Emergency Medicine,CIPROFLOXACIN HCL,CIPROFLOXACIN HCL,15,61.49
Melanie Aaberg,1801093968,TRENTON,MI,Emergency Medicine,DONEPEZIL HCL,DONEPEZIL HCL,12,125.20
Melanie Aaberg,1801093968,TRENTON,MI,Emergency Medicine,HYDROCODONE-ACETAMINOPHEN,HYDROCODONE/ACETAMINOPHEN,39,251.52
Melanie Aaberg,1801093968,TRENTON,MI,Emergency Medicine,IBUPROFEN,IBUPROFEN,16,67.06


Because of the size of each year's txt file (~3 GB), I decided to read each year's file in chunks, select the desired data using `wrangle_data`, and then write it to an SQL file accordingly. I wrote each year's data to a different table in the SQL file I used to avoid naming conflicts.

**If you run this code to verify my output, note that the following cell takes a very long time to run.**

In [5]:
data_files = []
sqlite_file = 'MedicareData.sqlite'
rx_conn = sqla.create_engine('sqlite:///' + sqlite_file)
for year in years:
    fname = 'MedicareData/PartD_Prescriber_PUF_NPI_Drug_{}.txt'.format(year)
    table = 'rx20' + str(year)
    data_files.append((fname, table))

for fname, table in data_files:
    # create file iterator
    MedData = pd.read_csv(fname, chunksize = 10000, low_memory= False, delimiter="\t")
    
    # intialize table
    MedChunk = next(MedData)
    wrangle_data(MedChunk).to_sql(table, rx_conn, if_exists = "replace")
    
    # read in the rest of the data
    for MedChunk in MedData:
        wrangle_data(MedChunk).to_sql(table, rx_conn, if_exists = "append")